In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master spark://iahlvldpt002.logistics.corp:7077 --packages com.datastax.spark:spark-cassandra-connector_2.12:3.0.0  pyspark-shell'

from pyspark import SparkContext
from pyspark.conf import SparkConf

from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import desc, max,count, avg
from pyspark.sql.functions import col, concat, udf, upper


import ceja




spark = SparkSession.builder \
    .appName('SparkCassandraApp_Fuzzy_address') \
    .config('spark.cassandra.connection.host', 'avengers-cassandra.kac.logistics.corp') \
    .config('spark.cassandra.connection.port', '9042') \
    .config('spark.cassandra.auth.username','mlai') \
    .config('spark.cassandra.auth.password','mlai') \
    .config('spark.cassandra.output.consistency.level','ONE') \
    .config('spark.cassandra.connection.timeoutBeforeCloseMS','12000000') \
    .config('spark.cassandra.read.timeoutMS','12000000') \
    .config('spark.cassandra.connection.timeoutMS','12000000') \
    .config('spark.cores.max','6') \
    .config('spark.sql.catalog.myCatalog','com.datastax.spark.connector.datasource.CassandraCatalog') \
    .master('spark://iahlvldpt002.logistics.corp:7077') \
    .getOrCreate()


     #    .config('spark.executor.memory','15g') \
     #    .config('spark.executor.cores','9') \
#adf= df.filter(df.state=='CA').withColumn("city_meta", ceja.metaphone(df.city)).withColumn("city_rev", reverse_value(df.city)) \
#       .withColumn("city_rev_meta", ceja.metaphone(reverse_value(df.city)) ).withColumn("cityu", upper(df.city))


In [2]:
@udf("string")
def reverse_value(value):
    return value[::-1]

sqlContext = SQLContext(spark)
cdf = sqlContext \
  .read \
  .format('org.apache.spark.sql.cassandra') \
  .options(table='addresses', keyspace='ml_ai_forecast_model') \
  .load()

df = cdf.select(upper("city").alias("city"),upper("state").alias("state"),upper("country").alias("country")).filter(cdf.state=="CA").distinct()

In [3]:
adf= df.filter(df.state=='CA').withColumn("city_meta", ceja.metaphone( df.city )).withColumn("city_rev", reverse_value( df.city ) )\
       .withColumn("city_rev_meta", ceja.metaphone(reverse_value( df.city  )) ).withColumn("cityu", upper(df.city)) 
       
adf.show()

+--------------+-----+-------------+---------+--------------+-------------+--------------+
|          city|state|      country|city_meta|      city_rev|city_rev_meta|         cityu|
+--------------+-----+-------------+---------+--------------+-------------+--------------+
|SPRINGS VALLEY|   CA|UNITED STATES|SPRNKS FL|YELLAV SGNIRPS|   YLF SKNRPS|SPRINGS VALLEY|
|      VAN NUYS|   CA|UNITED STATES|    FN NS|      SYUN NAV|       SYN NF|      VAN NUYS|
|      FAWNSKIN|   CA|UNITED STATES|    FNSKN|      NIKSNWAF|       NKSNWF|      FAWNSKIN|
|  PALM DESERTQ|   CA|UNITED STATES|PLM TSRTK|  QTRESED MLAP|    KTRST MLP|  PALM DESERTQ|
|   GTANITE BAY|   CA|UNITED STATES|   KTNT B|   YAB ETINATG|     YB ETNTK|   GTANITE BAY|
|    BURLINGHAM|   CA|UNITED STATES|  BRLNKHM|    MAHGNILRUB|       MKNLRB|    BURLINGHAM|
|  LAGUNANIGUEL|   CA|UNITED STATES|   LKNNKL|  LEUGINANUGAL|       LJNNKL|  LAGUNANIGUEL|
|   TOLUGA LAKE|   CA|UNITED STATES|   TLK LK|   EKAL AGULOT|     EKL AKLT|   TOLUGA LAKE|

In [4]:
#Loading Master Address
from pyspark.sql import Row, functions as F
from pyspark.sql.types import *


mdf = sqlContext \
  .read \
  .format('org.apache.spark.sql.cassandra') \
  .options(table='master_address_v2', keyspace='ml_ai_forecast_model') \
  .load()
#mdf.show()
# & ( (mdf.m_city=='San Francisco') |  (mdf.m_city=='San Fernando')  )

meta_mdf = mdf.filter( (mdf.m_state=='CA') ) \
              .withColumnRenamed("country", "m_country")\
              .withColumn("m_city_meta", ceja.metaphone(  upper(mdf.m_city)  )) \
              .withColumn("m_city_rev", reverse_value(  upper(mdf.m_city)   )).withColumn("m_city_rev_meta", ceja.metaphone(reverse_value( upper(mdf.m_city)  )  ) ) \
              .withColumn("m_cityu", upper(mdf.m_city)) \
              .withColumn("m_city_len", F.length(mdf.m_city)) 
meta_mdf.show()

+----------------+-------+-------------+-----------+----------------+---------------+----------------+----------+
|          m_city|m_state|    m_country|m_city_meta|      m_city_rev|m_city_rev_meta|         m_cityu|m_city_len|
+----------------+-------+-------------+-----------+----------------+---------------+----------------+----------+
|       Camarillo|     CA|United States|       KMRL|       OLLIRAMAC|          OLRMK|       CAMARILLO|         9|
|      Wrightwood|     CA|United States|       RTWT|      DOOWTHGIRW|           T0JR|      WRIGHTWOOD|        10|
|    Coulterville|     CA|United States|     KLTRFL|    ELLIVRETLUOC|        ELFRTLK|    COULTERVILLE|        12|
|    Big Oak Flat|     CA|United States|  BK OK FLT|    TALF KAO GIB|       TLF K JB|    BIG OAK FLAT|        12|
|            Etna|     CA|United States|        ETN|            ANTE|            ANT|            ETNA|         4|
|        Adelanto|     CA|United States|      ATLNT|        OTNALEDA|          OTNLT|   

In [5]:
#Join two datasets and perform similarity
from pyspark.sql.functions import substring as py_substring
#from pyspark.sql.column import Column
#                  .withColumn("sim_city_adj", ceja.jaro_similarity(  join_st_df['cityu'].substr(  1, join_st_df.m_city_len ) , join_st_df.m_cityu)) \
@udf("string")
def s_left(s, m):  
      if len(s) > (m+3) :
        return s[:(m+3)]
      else: 
        return s
    

join_st_df = adf.join(meta_mdf, adf.state==meta_mdf.m_state)

sim_df = join_st_df.select("cityu","city_meta","city_rev","city_rev_meta","state","country","m_cityu", "m_city_meta","m_city_rev","m_city_rev_meta","m_city_len") \
                   .withColumn("sim_city", ceja.jaro_similarity(join_st_df.cityu, join_st_df.m_cityu)) \
                   .withColumn("sim_adj_city", ceja.jaro_similarity(s_left("cityu", "m_city_len"), join_st_df.m_cityu)) \
                   .withColumn("jw_sim", ceja.jaro_winkler_similarity(join_st_df.cityu , join_st_df.m_cityu )) \
                   .withColumn("sim_city_rev", ceja.jaro_similarity(s_left("city_rev", "m_city_len"), join_st_df.m_city_rev)) \
                   .withColumn("jw_sim_rev", ceja.jaro_winkler_similarity(join_st_df.city_rev, join_st_df.m_city_rev )) \
                   .withColumn("sim_city_meta", ceja.jaro_similarity(join_st_df.city_meta, join_st_df.m_city_meta)) \
                   .withColumn("sim_city_rev_meta", ceja.jaro_similarity(join_st_df.city_rev_meta, join_st_df.m_city_rev_meta))  
#                   .withColumn("adj_city",s_left("cityu", "m_city_len") )

#sim_df.withColumn("adj_city",s_left("cityu","adj_len") ).show()
#                   .withColumn("adj_len", s_end(F.length(join_st_df.cityu),  F.length(join_st_df.m_cityu) ) )
#s_end(F.length(join_st_df.cityu) )
#                   .withColumn("test2", F.length(join_st_df.cityu)  )  \
#                   .withColumn("test", join_st_df.cityu )
#                   .withColumn("test", join_st_df.cityu.substr(1,s_end(F.length(join_st_df.cityu),  F.length(join_st_df.m_cityu) )  ))  \
sim_df.show()
#                  .withColumn("city_adj", pleft(join_st_df.cityu,join_st_df.m_city,join_st_df.m_city_len))

+---------+---------+---------+-------------+-----+-------------+-------------------+------------+-------------------+---------------+----------+----------+------------+----------+------------+----------+-------------+-----------------+
|    cityu|city_meta| city_rev|city_rev_meta|state|      country|            m_cityu| m_city_meta|         m_city_rev|m_city_rev_meta|m_city_len|  sim_city|sim_adj_city|    jw_sim|sim_city_rev|jw_sim_rev|sim_city_meta|sim_city_rev_meta|
+---------+---------+---------+-------------+-----+-------------+-------------------+------------+-------------------+---------------+----------+----------+------------+----------+------------+----------+-------------+-----------------+
|MILLVILLE|     MLFL|ELLIVLLIM|        ELFLM|   CA|UNITED STATES|SOUTH SAN FRANCISCO|S0 SN FRNSSK|OCSICNARF NAS HTUOS| OKSKNRF NS HTS|        19|       0.0|         0.0|       0.0|  0.38791424|0.38791424|   0.44444445|       0.42380953|
|MILLVILLE|     MLFL|ELLIVLLIM|        ELFLM|   CA|U

In [6]:
#get max values from 7
import builtins as py_builtin

@udf("double")
def max_value(v1,v2,v3,v4,v5):
    return  py_builtin.max([v1,v2,v3,v4,v5])

max_sim = sim_df.filter( (sim_df.sim_city > 0.8) | (sim_df.sim_city_rev > 0.8)   ).sort(desc("sim_city")) \
      .select("cityu","state","country","m_cityu","sim_city","sim_city_rev","sim_adj_city", "sim_city_meta","jw_sim") \
      .withColumn("max_score",max_value(sim_df.sim_city, sim_df.sim_city_rev,  sim_df.sim_adj_city,sim_df.sim_city_meta,  sim_df.jw_sim) )  
max_sim.show()

+--------------+-----+-------------+--------------+--------+------------+------------+-------------+------+---------+
|         cityu|state|      country|       m_cityu|sim_city|sim_city_rev|sim_adj_city|sim_city_meta|jw_sim|max_score|
+--------------+-----+-------------+--------------+--------+------------+------------+-------------+------+---------+
|        NELSON|   CA|UNITED STATES|        NELSON|     1.0|         1.0|         1.0|          1.0|   1.0|      1.0|
|       ARTESIA|   CA|UNITED STATES|       ARTESIA|     1.0|         1.0|         1.0|          1.0|   1.0|      1.0|
|     VACAVILLE|   CA|UNITED STATES|     VACAVILLE|     1.0|         1.0|         1.0|          1.0|   1.0|      1.0|
|     ALTAVILLE|   CA|UNITED STATES|     ALTAVILLE|     1.0|         1.0|         1.0|          1.0|   1.0|      1.0|
|   WEAVERVILLE|   CA|UNITED STATES|   WEAVERVILLE|     1.0|         1.0|         1.0|          1.0|   1.0|      1.0|
|CAMP PENDLETON|   CA|UNITED STATES|CAMP PENDLETON|     

In [7]:

#row_number
from pyspark.sql.window import Window   

fz_df = max_sim.select("cityu", "state","m_cityu", "max_score", "sim_city" ,"sim_city_rev","sim_adj_city", "sim_city_meta","jw_sim"                        
                        ,F.row_number().over( Window.partitionBy("cityu").orderBy(desc("max_score"))).alias("rowNum") ) \
               .filter(max_sim.max_score > 0.8 ).sort("cityu")
fz_df.select("cityu", "state","m_cityu", "max_score").filter(fz_df.rowNum==1).show(1000)


+--------------------+-----+--------------------+------------------+
|               cityu|state|             m_cityu|         max_score|
+--------------------+-----+--------------------+------------------+
|           ADELANTO |   CA|            ADELANTO|0.9444444179534912|
|              CARSON|   CA|              CARSON|               1.0|
|             CASTAIC|   CA|             CASTAIC|               1.0|
|               CHINO|   CA|               CHINO|               1.0|
|    CITY OF INDUSTRY|   CA|    CITY OF INDUSTRY|               1.0|
|             FONTANA|   CA|             FONTANA|               1.0|
|              IRVINE|   CA|              IRVINE|               1.0|
|              NOVATO|   CA|              NOVATO|               1.0|
|            SAN JOSE|   CA|            SAN JOSE|               1.0|
|          , VALENCIA|   CA|            VALENCIA|               1.0|
|   1391SAN FRANCISCO|   CA|       SAN FRANCISCO|               1.0|
|       200 YUDA CITY|   CA|      

In [8]:
#fz_df.select("cityu", "state","m_cityu", "max_score").filter(f (z_df.rowNum==1) & (fz_df.cityu="ARLINGTON") ).show()
fz_df.filter((fz_df.rowNum==1) & ( (fz_df.cityu=="ANAHEIN HILLS,") | (fz_df.cityu=="ARLINGTON") | (fz_df.cityu=="ACACIA ONTARIO") ) ).show()
#

+--------------+-----+-----------+------------------+----------+------------+------------+-------------+----------+------+
|         cityu|state|    m_cityu|         max_score|  sim_city|sim_city_rev|sim_adj_city|sim_city_meta|    jw_sim|rowNum|
+--------------+-----+-----------+------------------+----------+------------+------------+-------------+----------+------+
|ACACIA ONTARIO|   CA|    ONTARIO|0.8999999761581421|0.54761904|         0.9|   0.5761905|          0.0|0.54761904|     1|
|ANAHEIN HILLS,|   CA|CHINO HILLS|0.8203462958335876| 0.7833093|   0.8203463|   0.7833093|   0.75277776| 0.7833093|     1|
|     ARLINGTON|   CA| FARMINGTON|0.8962963223457336| 0.8962963|   0.8962963|   0.8962963|    0.8095238| 0.8962963|     1|
+--------------+-----+-----------+------------------+----------+------------+------------+-------------+----------+------+



In [9]:
fz_df.select("cityu", "state","m_cityu", "max_score","sim_city").filter(fz_df.rowNum==1).show(1000)
fz_df.registerTempTable('fuzzy_city_mapping')


+--------------------+-----+--------------------+------------------+----------+
|               cityu|state|             m_cityu|         max_score|  sim_city|
+--------------------+-----+--------------------+------------------+----------+
|           ADELANTO |   CA|            ADELANTO|0.9444444179534912|0.93333334|
|              CARSON|   CA|              CARSON|               1.0|0.95238096|
|             CASTAIC|   CA|             CASTAIC|               1.0| 0.9583333|
|               CHINO|   CA|               CHINO|               1.0| 0.9444444|
|    CITY OF INDUSTRY|   CA|    CITY OF INDUSTRY|               1.0|0.89705884|
|             FONTANA|   CA|             FONTANA|               1.0| 0.9583333|
|              IRVINE|   CA|              IRVINE|               1.0|0.95238096|
|              NOVATO|   CA|              NOVATO|               1.0|0.95238096|
|            SAN JOSE|   CA|            SAN JOSE|               1.0| 0.8796296|
|          , VALENCIA|   CA|            

In [10]:
sqlContext.sql("select * from fuzzy_city_mapping").show()

+-----------------+-----+-------------------+------------------+----------+------------+------------+-------------+----------+------+
|            cityu|state|            m_cityu|         max_score|  sim_city|sim_city_rev|sim_adj_city|sim_city_meta|    jw_sim|rowNum|
+-----------------+-----+-------------------+------------------+----------+------------+------------+-------------+----------+------+
|        ADELANTO |   CA|           ADELANTO|0.9444444179534912|0.93333334|  0.93333334|  0.93333334|    0.9444444|0.93333334|     1|
|        ADELANTO |   CA|              LATON|0.8083333373069763| 0.5222222|  0.80833334|        0.55|    0.7222222| 0.5222222|     3|
|        ADELANTO |   CA|             DELANO|0.8888888955116272|0.75555557|   0.8888889|   0.7777778|    0.8333333|0.75555557|     2|
|           CARSON|   CA|            CRESTON|0.9333333373069763| 0.8095238|   0.8095238|   0.8095238|   0.93333334| 0.8095238|     2|
|           CARSON|   CA|         KIT CARSON|0.899999976158142

In [11]:
#SAN FRANCISCO
sqlContext.sql("select * from fuzzy_city_mapping where m_cityu='SAN FRANCISCO' and rowNum==1 ").show(1000)


+--------------------+-----+-------------+------------------+----------+------------+------------+-------------+----------+------+
|               cityu|state|      m_cityu|         max_score|  sim_city|sim_city_rev|sim_adj_city|sim_city_meta|    jw_sim|rowNum|
+--------------------+-----+-------------+------------------+----------+------------+------------+-------------+----------+------+
|   1391SAN FRANCISCO|   CA|SAN FRANCISCO|               1.0|0.92156863|      0.9375|  0.89102566|          1.0|0.92156863|     1|
|   371BSAN FRANCISCO|   CA|SAN FRANCISCO|0.9666666388511658|0.92156863|      0.9375|  0.89102566|   0.96666664|0.92156863|     1|
|  3A10 SAN FRANCISCO|   CA|SAN FRANCISCO|               1.0|0.83048433|      0.9375|   0.7536422|          1.0|0.83048433|     1|
|         AN FANCISCO|   CA|SAN FRANCISCO|0.9487179517745972|0.94871795|  0.94871795|  0.94871795|    0.8842593|0.94871795|     1|
|        AN FRANCISCO|   CA|SAN FRANCISCO|0.9743589758872986|  0.974359|    0.97435

In [12]:
sqlContext.sql("select count(*) from fuzzy_city_mapping where m_cityu='SAN FRANCISCO' and rowNum==1 ").show(1000)

+--------+
|count(1)|
+--------+
|     125|
+--------+

